In [66]:
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from Clasificador import Clasificador

from abc import ABC, abstractmethod


class Individuo:
    def __init__(self, reglas: np.ndarray):
        self.reglas = reglas

    @staticmethod
    def crea_con_reglas_aleatorias(max_reglas: int, longitud_reglas: int):
        while True:
            reglas = np.random.randint(
                2, size=(np.random.randint(1, max_reglas), longitud_reglas)
            )
            # Verificar si alguna fila tiene solo 0s o solo 1s
            if not np.any(np.all(reglas == 0, axis=1)) and not np.any(
                np.all(reglas == 1, axis=1)
            ):
                break

        return Individuo(reglas)

    def clasifica(self, dato: np.ndarray) -> int | None:
        votos = [0, 0]

        for regla in self.reglas:
            regla_activada = True

            for dato_bit, regla_bit in zip(dato, regla[:-1]):
                if dato_bit == 1 and regla_bit != 1:
                    regla_activada = False
                    break

            if regla_activada:
                votos[regla[-1]] += 1

        if sum(votos) == 0:
            return None
        elif votos[0] > votos[1]:
            return 0
        elif votos[0] < votos[1]:
            return 1
        else:
            return -1

    def fitness(self, datos_codificados: np.ndarray) -> float:
        aciertos = 0

        for dato in datos_codificados:
            prediccion = self.clasifica(dato[:-1])

            if dato[-1] == prediccion:
                aciertos += 1

        return aciertos / datos_codificados.shape[0]

    # metodo prototipo
    def copia(self):
        return Individuo(np.copy(self.reglas))

class EstrategiaMutacion(ABC):
    @abstractmethod
    def aplicar_mutacion(self, individuos: List[Individuo]) -> List[Individuo]:
        pass

class EstrategiaCruce(ABC):
    @abstractmethod
    def aplicar_cruce(self, individuos: List[Individuo]) -> List[Individuo]:
        pass

class CruceInterReglas(EstrategiaCruce):
    def aplicar_cruce(self, individuos: List[Individuo]) -> List[Individuo]:
        descendientes = []

        for _ in range(len(individuos) // 2):
            progenitor1, progenitor2 = np.random.choice(individuos, size=2, replace=False)
            punto_cruce1, punto_cruce2 = np.random.randint(min(len(progenitor1.reglas), len(progenitor2.reglas)), size=2)

            nueva_reglas1 = np.vstack((progenitor1.reglas[:punto_cruce1], progenitor2.reglas[punto_cruce2:]))
            nueva_reglas2 = np.vstack((progenitor1.reglas[punto_cruce1:], progenitor2.reglas[:punto_cruce2]))

            descendiente1 = Individuo(nueva_reglas1)
            descendiente2 = Individuo(nueva_reglas2)

            descendientes.extend([descendiente1, descendiente2])

        return descendientes

class CruceIntraReglas(EstrategiaCruce):
    def aplicar_cruce(self, individuos: List[Individuo]) -> List[Individuo]:
        descendientes = []

        for _ in range(len(individuos) // 2):
            progenitor1, progenitor2 = np.random.choice(individuos, size=2, replace=False)

            # Seleccionar una regla común entre ambos progenitores
            regla_comun = np.random.randint(min(len(progenitor1.reglas), len(progenitor2.reglas)))

            # Seleccionar un punto de corte aleatorio
            punto_cruce = np.random.randint(len(progenitor1.reglas[regla_comun]))

            # Intercambiar el material genético en la regla común
            nueva_regla1 = np.copy(progenitor1.reglas)
            nueva_regla2 = np.copy(progenitor2.reglas)

            nueva_regla1[regla_comun, :punto_cruce] = progenitor2.reglas[regla_comun, :punto_cruce]
            nueva_regla2[regla_comun, :punto_cruce] = progenitor1.reglas[regla_comun, :punto_cruce]

            descendiente1 = Individuo(nueva_regla1)
            descendiente2 = Individuo(nueva_regla2)

            descendientes.extend([descendiente1, descendiente2])

        return descendientes

class MutacionEstandar(EstrategiaMutacion):
    def aplicar_mutacion(self, individuos: List[Individuo]) -> List[Individuo]:
        descendientes = []
        probabilidad_mutacion = 1 / individuos[0].reglas.size

        for individuo in individuos:
            mutado = individuo.copia()
            punto_mutacion = np.random.randint(len(mutado.reglas))

            for i in range(len(mutado.reglas[punto_mutacion])):
                if np.random.rand() < probabilidad_mutacion:
                    mutado.reglas[punto_mutacion, i] ^= 1

            descendientes.append(mutado)

        return descendientes


class Poblacion:
    def __init__(self, individuos: List[Individuo]):
        self._individuos = individuos

        self._promedio_fitness = -1
        self._mejor_fitness = -1
        self._mejor_individuo = None

    def fitness(self, datos_codificados: np.ndarray) -> List[Tuple[Individuo, float]]:
        fitness_poblacion = []
        suma_fitness_poblacion = 0

        mejor_fitness = -1
        mejor_individuo = None

        for individuo in self._individuos:
            fitness_individuo = individuo.fitness(datos_codificados)

            fitness_poblacion.append((individuo, fitness_individuo))
            suma_fitness_poblacion += fitness_individuo

            if fitness_individuo > mejor_fitness:
                mejor_fitness = fitness_individuo
                mejor_individuo = individuo

        self._promedio_fitness = suma_fitness_poblacion / len(self._individuos)
        self._mejor_fitness = mejor_fitness
        self._mejor_individuo = mejor_individuo

        return fitness_poblacion

    def individuos(self) -> List[Individuo]:
        return self._individuos

    def mejor_individuo(self) -> Individuo:
        return self._mejor_individuo

    def mejor_fitness(self) -> float:
        return self._mejor_fitness

    def promedio_fitness(self) -> float:
        return self._promedio_fitness


class CodificadorBinario:
    def __init__(self, datos: pd.DataFrame):
        self._n_bits, self._codificacion = self._init_codificacion(datos)

    def _init_codificacion(
        self, datos: pd.DataFrame
    ) -> Tuple[int, Dict[str, Dict[str, List[int]]]]:
        atributos = datos.columns[:-1]
        objetivo = datos.columns[-1]

        codificacion = {}
        n_bits = 0  # cantidad de bits necesarios para codificar una muestra

        for atributo in atributos:
            valores = sorted(datos[atributo].astype(str).unique())
            codificacion[atributo] = {}

            for i, valor in enumerate(valores):
                codigo = [0] * len(valores)
                codigo[i] = 1

                codificacion[atributo][valor] = codigo

            n_bits += len(valores)

        # Columna de clase, tiene una codificación con un solo bit
        codificacion[objetivo] = {"0": [0], "1": [1], "+": [1], "-": [0]}
        n_bits += 1

        return n_bits, codificacion

    def codificacion(self) -> Dict[str, Dict[str, List[int]]]:
        return self._codificacion

    def codifica_datos(self, datos: pd.DataFrame) -> np.ndarray:
        filas_codificadas = []

        for _, fila in datos.iterrows():
            fila_codificada = []

            for columna, valor in fila.items():
                fila_codificada.extend(self._codificacion[columna][str(valor)])

            filas_codificadas.append(fila_codificada)

        return np.array(filas_codificadas)

    def n_bits(self) -> int:
        return self._n_bits


class AlgoritmoGenetico(Clasificador):
    def __init__(
        self,
        tamano_poblacion: int,
        n_generaciones: int,
        max_reglas: int,
        porcentaje_elitismo: float,
        cruces: List[EstrategiaCruce],
        mutaciones: List[EstrategiaMutacion]
    ):
        self._max_reglas = max_reglas
        self._n_elitistas = int(np.ceil(tamano_poblacion * porcentaje_elitismo))
        self._n_generaciones = n_generaciones
        self._tamano_poblacion = tamano_poblacion
        self._cruces = cruces
        self._mutaciones = mutaciones

        self._codificador = None
        self._generaciones = []
        self._clase_mas_probable = 1

    def entrenamiento(
        self, datosTrain: pd.DataFrame, nominalAtributos: List[bool], diccionario: Dict
    ):
        # Crea codificacion -> esto podria ser a traves de diccionario
        self._codificador = CodificadorBinario(datosTrain)

        # Crear primera generacion
        self._init_generaciones()
        datos_codificados = self._codificador.codifica_datos(datosTrain)

        for _ in range(self._n_generaciones):
            poblacion = self._generaciones[-1]

            # calcular fitness de la poblacion
            fitness_poblacion = poblacion.fitness(datos_codificados)

            # utilizar elitismo
            elite = self._selecciona_elite(fitness_poblacion)

            # selecciona progenitores
            progenitores = self._selecciona_progenitores(fitness_poblacion)

            # operadores geneticos
            descendientes = self._operador_cruce(progenitores)
            descendientes = self._operador_mutacion(descendientes)

            nuevos_individuos = elite + descendientes
            nueva_poblacion = Poblacion(nuevos_individuos)

            print(f"Nueva poblacion, mejor: {poblacion.mejor_fitness()}, promedio: {poblacion.promedio_fitness()}")

            self._generaciones.append(nueva_poblacion)

        self._generaciones[-1].fitness(datos_codificados)
        self._encuentra_mejor_generacion(datos_codificados)

    def _init_generaciones(self) -> Poblacion:
        individuos = []

        # Generar `tamano_poblacion` individuos con reglas aleatorias
        for _ in range(self._tamano_poblacion):
            individuos.append(
                Individuo.crea_con_reglas_aleatorias(
                    max_reglas=self._max_reglas,
                    longitud_reglas=self._codificador.n_bits(),
                )
            )

        self._generaciones.append(Poblacion(individuos))

    def _selecciona_elite(
        self, fitness_poblacion: List[Tuple[Individuo, float]]
    ) -> List[Individuo]:
        # Ordenar la población según la aptitud (mayor aptitud primero)
        poblacion_ordenada = [
            individuo
            for individuo, _ in sorted(
                fitness_poblacion, key=lambda x: x[1], reverse=True
            )
        ]

        # Seleccionar a los mejores individuos (élite)
        elite = poblacion_ordenada[: self._n_elitistas]

        return elite

    def _selecciona_progenitores(
        self, fitness_poblacion: List[Tuple[Individuo, float]]
    ) -> List[Individuo]:
        n_progenitores = self._tamano_poblacion - self._n_elitistas
        suma_fitness_poblacion = sum(
            fitness_individuo for _, fitness_individuo in fitness_poblacion
        )

        # Normalizar la aptitud para convertirla en probabilidades
        probabilidad_seleccion = [
            fitness_individuo / suma_fitness_poblacion
            for _, fitness_individuo in fitness_poblacion
        ]

        # Utilizar np.random.choice para seleccionar progenitores
        progenitores_indices = np.random.choice(
            np.arange(len(fitness_poblacion)),
            size=n_progenitores,
            p=probabilidad_seleccion,
        )
        progenitores = [fitness_poblacion[i][0].copia() for i in progenitores_indices]

        return progenitores

    def _operador_cruce(self, individuos: List[Individuo]) -> List[Individuo]:
        descendientes = individuos
        
        for cruce in self._cruces:
            descendientes = cruce.aplicar_cruce(descendientes)

        return descendientes

    def _operador_mutacion(self, individuos: List[Individuo]) -> List[Individuo]:
        descendientes = individuos
        
        for mutacion in self._mutaciones:
            descendientes = mutacion.aplicar_mutacion(descendientes)

        return descendientes

    def _encuentra_mejor_generacion(self, datos_codificados: np.ndarray):
        mejor_fitness = -1
        mejor_generacion = None
    
        for generacion in self._generaciones:
            mejor_individuo = generacion.mejor_individuo()
            fitness_individuo = mejor_individuo.fitness(datos_codificados)

            if fitness_individuo > mejor_fitness:
                mejor_fitness = fitness_individuo
                mejor_generacion = generacion

        self._mejor_generacion = mejor_generacion
            

    def representacion_condicional(self, individuo: Individuo) -> List[str]:
        reglas = individuo.reglas
        representaciones = [self._interpretar_regla(regla, self._codificador.codificacion()) for regla in reglas]

        return representaciones

    def _interpretar_regla(self, regla_binaria: List[int], codificacion) -> str:
        output = "\nIF "
        posicion_actual = 0
        
        for atributo, valores in codificacion.items():
            condiciones = [
                f"({atributo} == {list(valores.keys())[i_bit]})"
                for i_bit, bit in enumerate(regla_binaria[posicion_actual:posicion_actual + len(valores)])
                if bit == 1
            ]
    
            if condiciones:
                output += "\t AND " if output != "\nIF " else "\t"
                output += " OR ".join(condiciones)
                output += "\n"
    
            posicion_actual += len(valores)
    
        output += f"THEN Class == {regla_binaria[-1]}"
    
        return output

    def clasifica(self, datosTest: pd.DataFrame, nominalAtributos: List[bool], diccionario: Dict):
        predicciones = []
        datos_test_codificados = self._codificador.codifica_datos(datosTest)

        X_test = datos_test_codificados[:,:-1]

        for dato in X_test:
            prediccion = self._mejor_generacion.mejor_individuo().clasifica(dato)

            if prediccion == -1 or prediccion == None:
                prediccion = self._clase_mas_probable

            predicciones.append(prediccion)

        return np.array(predicciones)

In [67]:
import pandas as pd

df_titanic = pd.read_csv("titanic.csv")

In [68]:
clasificador = AlgoritmoGenetico(
    tamano_poblacion=100,
    n_generaciones=5,
    max_reglas=5,
    porcentaje_elitismo=0.05,
    cruces=[CruceInterReglas(), CruceIntraReglas()],
    mutaciones=[MutacionEstandar()]
)

In [69]:
clasificador.entrenamiento(datosTrain=df_titanic, nominalAtributos=[], diccionario={})

Nueva poblacion, mejor: 0.44263862332695986, promedio: 0.13681644359464626
Nueva poblacion, mejor: 0.5544933078393881, promedio: 0.24240492882940295
Nueva poblacion, mejor: 0.5917782026768642, promedio: 0.304140834733569
Nueva poblacion, mejor: 0.5917782026768642, promedio: 0.3872182629352802
Nueva poblacion, mejor: 0.5917782026768642, promedio: 0.39232670877030335


In [73]:
for i, regla in enumerate(clasificador.representacion_condicional(clasificador._mejor_generacion.mejor_individuo())):
    print(regla, "\n", clasificador._mejor_generacion.mejor_individuo().reglas[i])


IF 	(Pclass == 1) OR (Pclass == 2) OR (Pclass == 3)
	 AND (Sex == female) OR (Sex == male)
	 AND (Age == 0) OR (Age == 11) OR (Age == 15) OR (Age == 3) OR (Age == 4) OR (Age == 5) OR (Age == 7) OR (Age == 8)
THEN Class == 0 
 [1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0]

IF 	(Pclass == 1) OR (Pclass == 3)
	 AND (Sex == male)
	 AND (Age == 0) OR (Age == 1) OR (Age == 10) OR (Age == 15) OR (Age == 4) OR (Age == 5) OR (Age == 6) OR (Age == 7) OR (Age == 8)
THEN Class == 0 
 [1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 0]

IF 	(Pclass == 1) OR (Pclass == 2) OR (Pclass == 3)
	 AND (Sex == female) OR (Sex == male)
	 AND (Age == 0) OR (Age == 1) OR (Age == 10) OR (Age == 12) OR (Age == 13) OR (Age == 15) OR (Age == 6) OR (Age == 7) OR (Age == 8) OR (Age == 9)
THEN Class == 0 
 [1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 0]

IF 	(Pclass == 1) OR (Pclass == 2) OR (Pclass == 3)
	 AND (Sex == female) OR (Sex == male)
	 AND (Age == 0) OR (Age == 1) OR (Age == 12) OR (Age == 13) OR (Age == 14) OR (A

In [6]:
preds[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0])

In [7]:
codificacion = clasificador._codificador._codificacion

In [8]:
actual = 0
columna_actual = 0

for column, values in codificacion.items():
    print("indices: ", actual, actual + len(values))
    print("columna actual: ", columna_actual)

    actual += len(values)
    columna_actual += 1

indices:  0 3
columna actual:  0
indices:  3 5
columna actual:  1
indices:  5 21
columna actual:  2
indices:  21 25
columna actual:  3


In [9]:
codificacion

{'Pclass': {'1': [1, 0, 0], '2': [0, 1, 0], '3': [0, 0, 1]},
 'Sex': {'female': [1, 0], 'male': [0, 1]},
 'Age': {'0': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '10': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '11': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '12': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '13': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '14': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '15': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  '2': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  '3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  '4': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  '5': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  '6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  '7': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  '8': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  '9': [0, 0, 0, 0, 0, 0, 0

clasificador._mejor_generacion.mejor_individuo().reglas[0]

In [10]:
def interpretar_regla(regla_binaria: List[int], codificacion) -> str:
    texto_plano = "\nIF "
    
    atr_idx = 0
    atr_values = codificacion

    for attr, values in atr_values.items():
        interpretation += f"'{attr}'="

        for i in range(len(values)-1):
            if regla_binaria[atr_idx] == 1:
                interpretation += f"{list(values.keys())[i]} OR "

            atr_idx += 1

        # Eliminar el último " OR " agregado
        interpretation = interpretation[:-4]
        interpretation += " AND "

    # Eliminar el último " AND " agregado
    interpretation = interpretation[:-5]

    interpretation += f" THEN 'Class'={regla_binaria[-1]}"

    return interpretation

individuo = clasificador._generaciones[-1].mejor_individuo()
reglas = individuo.reglas

interpretar_regla(reglas[0], codificacion)

"IF 'Pclass'=1 OR 2 AND 'S AND 'Age'=0 OR 11 OR 12 OR 14 OR 15 OR 2 OR 3 OR 5 OR 6 OR 7 OR 8 AND 'Class'=0 OR 1 OR + THEN 'Class'=1"

In [ ]:
https://github.com/search?q=algoritmogenetico%28clasificador%29&type=code

In [56]:
def interpretar_regla(regla_binaria: List[int], codificacion) -> str:
    output = "\nIF "
    posicion_actual = 0
    
    for atributo, valores in codificacion.items():
        condiciones = [
            f"({atributo} == {list(valores.keys())[i_bit]})"
            for i_bit, bit in enumerate(regla_binaria[posicion_actual:posicion_actual + len(valores)])
            if bit == 1
        ]

        if condiciones:
            output += "\t AND " if output != "\nIF " else "\t"
            output += " OR ".join(condiciones)
            output += "\n"

        posicion_actual += len(valores)

    output += f"THEN Class == {regla_binaria[-1]}"

    return output

individuo = clasificador._generaciones[-1].mejor_individuo()
reglas = individuo.reglas

print(interpretar_regla(reglas[0], codificacion))


IF 	(Pclass == 1) OR (Pclass == 2)
	 AND (Sex == female)
	 AND (Age == 1) OR (Age == 10) OR (Age == 12) OR (Age == 13) OR (Age == 14) OR (Age == 15) OR (Age == 3) OR (Age == 4) OR (Age == 5) OR (Age == 6) OR (Age == 7) OR (Age == 8) OR (Age == 9)
	 AND (Class == 0)
THEN Class == 1


In [33]:
def representacion_condicional(regla_binaria: List[int], codificacion):
    class bcolors:
        HEADER = '\033[95m'
        OKBLUE = '\033[94m'
        OKCYAN = '\033[96m'
        OKGREEN = '\033[92m'
        WARNING = '\033[93m'
        FAIL = '\033[91m'
        ENDC = '\033[0m'
        BOLD = '\033[1m'
        UNDERLINE = '\033[4m'


    returnstring = ""

    lst = []
    for key, val in codificacion.items():
        lst.append({v: k for k, v in val.items()})

    
    returnstring += f"\n{bcolors.BOLD}IF{bcolors.ENDC}  "
        
    comienzoAtr = 0
    firstAtr = True

    for numAtr, atrLength in enumerate(self.atrLens):
        x = []
        finAtr = comienzoAtr + atrLength
        
        for idx_bit, bit in enumerate(regla[comienzoAtr:finAtr]):
            if bit == 1:
                x.append(f"( {bcolors.UNDERLINE}{self.nombresAtributos[numAtr]}{bcolors.ENDC} == {bcolors.FAIL}{lst[numAtr][idx_bit]}{bcolors.ENDC} )")
        
        if len(x) > 0:
            if firstAtr:

                returnstring += "  "

                firstAtr = False
            else:
                returnstring += f"  {bcolors.OKGREEN}AND{bcolors.ENDC} "

            returnstring += f" {bcolors.OKBLUE}OR{bcolors.ENDC} ".join(x)
            returnstring += "\n"

        comienzoAtr = finAtr

    returnstring += f"{bcolors.BOLD}THEN{bcolors.ENDC} {regla[-1]}\n"
    
    return returnstring

print(representacion_condicional(reglas[0], codificacion))

TypeError: unhashable type: 'list'